In [ ]:
# Colab 환경에서
# from google.colab import drive
# drive.mount('/content/drive')

## 데이터 불러오기

In [8]:
from pathlib import Path

current_path = Path().cwd()
root = current_path.parent.parent
data = root / 'data'
src = root / 'src'
images = root / 'images'
print(root.absolute())

/home/yeeun/Sparta/Project1


In [ ]:
import pandas as pd

datasets = {
    'games_details': pd.DataFrame(),
    'ranking': pd.DataFrame(),
    'games': pd.DataFrame(),
    'players': pd.DataFrame(),
    'teams': pd.DataFrame()
}

pandas_kwargs = {
    'games_details': {
        'dtype': {'NICKNAME': str}
    },
    'ranking': {
        
    },
    'games': {
        
    },
    'players': {
        
    },
    'teams': {
        
    }
}

for name in datasets:
    datasets[name] = pd.read_csv(data / f'{name}.csv', **pandas_kwargs[name])

games_details = datasets['games_details']
games = datasets['games']
players = datasets['players']
ranking = datasets['ranking']
teams = datasets['teams']

In [25]:
ranking.head()

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN


## ranking 데이터 구조 분석

### ranking의 기본 정보

In [17]:
ranking.shape

(210342, 13)

In [18]:
ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210342 entries, 0 to 210341
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TEAM_ID        210342 non-null  int64  
 1   LEAGUE_ID      210342 non-null  int64  
 2   SEASON_ID      210342 non-null  int64  
 3   STANDINGSDATE  210342 non-null  object 
 4   CONFERENCE     210342 non-null  object 
 5   TEAM           210342 non-null  object 
 6   G              210342 non-null  int64  
 7   W              210342 non-null  int64  
 8   L              210342 non-null  int64  
 9   W_PCT          210342 non-null  float64
 10  HOME_RECORD    210342 non-null  object 
 11  ROAD_RECORD    210342 non-null  object 
 12  RETURNTOPLAY   3990 non-null    float64
dtypes: float64(2), int64(6), object(5)
memory usage: 20.9+ MB


### ranking의 고유값 체크

In [63]:
ranking.nunique()

TEAM_ID            30
LEAGUE_ID           1
SEASON_ID          41
STANDINGSDATE    7023
CONFERENCE          2
TEAM               34
G                  83
W                  74
L                  73
W_PCT             895
HOME_RECORD       800
ROAD_RECORD       758
RETURNTOPLAY        2
dtype: int64

### Null 값 체크

In [79]:
ranking.isna().mean() * 100

TEAM_ID           0.000000
LEAGUE_ID         0.000000
SEASON_ID         0.000000
STANDINGSDATE     0.000000
CONFERENCE        0.000000
TEAM              0.000000
G                 0.000000
W                 0.000000
L                 0.000000
W_PCT             0.000000
HOME_RECORD       0.000000
ROAD_RECORD       0.000000
RETURNTOPLAY     98.103089
dtype: float64

### 숫자형 컬럼 묘사

In [97]:
print(ranking['W_PCT'].describe().round(2))

count    210342.00
mean          0.49
std           0.19
min           0.00
25%           0.37
50%           0.50
75%           0.62
max           1.00
Name: W_PCT, dtype: float64


### 범주가 있는 컬럼

In [125]:
col = 'RETURNTOPLAY'
cnt = ranking[col].value_counts(dropna=False, ascending=False).to_frame("Count")
cnt['Proportion'] = ranking[col].value_counts(normalize=True, dropna=False, ascending=False).round(4) * 100
print(cnt)

               Count  Proportion
RETURNTOPLAY                    
NaN           206352       98.10
1.0             2394        1.14
0.0             1596        0.76


### 날짜 컬럼

In [126]:
team_id_cond = ranking['TEAM_ID'] == 1610612743
ranking[team_id_cond]['TEAM'].value_counts()

TEAM
Denver    7024
Name: count, dtype: int64

In [72]:
grouped = ranking.groupby(['TEAM_ID']).agg(DISTINCT_TEAM=('TEAM', 'nunique'))
grouped

,DISTINCT_TEAM
TEAM_ID,
1610612737,1
1610612738,1
1610612739,1
1610612740,2
1610612741,1
1610612742,1
1610612743,1
1610612744,1
1610612745,1


In [73]:
team_cond = (grouped[grouped['DISTINCT_TEAM'] > 1].index)
filtered = ranking[ranking['TEAM_ID'].isin(team_cond)].sort_values('STANDINGSDATE')
filtered
# ranking[team_cond]

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
92950,1610612746,0,22002,2003-10-01,West,L.A. Clippers,82,27,55,0.329,16-25,11-30,NaN
181543,1610612751,0,22002,2003-10-01,East,New Jersey,82,49,33,0.598,33-8,16-25,NaN
92947,1610612760,0,22002,2003-10-01,West,Seattle,82,40,42,0.488,25-16,15-26,NaN
181546,1610612740,0,22002,2003-10-01,East,New Orleans,82,47,35,0.573,29-12,18-23,NaN
181531,1610612740,0,22002,2003-10-02,East,New Orleans,82,47,35,0.573,29-12,18-23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,1610612740,0,22022,2022-12-21,West,New Orleans,30,18,12,0.600,12-4,6-8,NaN
11,1610612760,0,22022,2022-12-22,West,Oklahoma City,32,14,18,0.438,9-7,5-11,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN
4278,1610612751,0,22022,2022-12-22,East,Brooklyn,32,20,12,0.625,11-5,9-7,NaN


In [60]:
print(filtered.groupby(['TEAM_ID']).agg(DISTINCT_TEAM=('TEAM', 'unique')))

                                       DISTINCT_TEAM
TEAM_ID                                             
1610612740  [New Orleans, New Orleans/Oklahoma City]
1610612746              [L.A. Clippers, LA Clippers]
1610612751                    [New Jersey, Brooklyn]
1610612760                  [Seattle, Oklahoma City]


In [62]:
seattle = ranking['TEAM'] == 'Seattle'
ranking[seattle]['TEAM_ID'].value_counts()

TEAM_ID
1610612760    1831
Name: count, dtype: int64

### NULL값 조사

In [82]:
target_team = 'Milwaukee'
target_date = '2020-03-11'

cond = (ranking['TEAM'] == target_team) & (ranking['STANDINGSDATE'] >= target_date)
ranking[cond].sort_values('STANDINGSDATE')

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
30659,1610612749,0,22019,2020-03-11,East,Milwaukee,65,53,12,0.815,28-3,25-9,NaN
30644,1610612749,0,22019,2020-03-12,East,Milwaukee,65,53,12,0.815,28-3,25-9,1.0
30629,1610612749,0,22019,2020-03-13,East,Milwaukee,65,53,12,0.815,28-3,25-9,1.0
30614,1610612749,0,22019,2020-03-14,East,Milwaukee,65,53,12,0.815,28-3,25-9,1.0
30599,1610612749,0,22019,2020-03-15,East,Milwaukee,65,53,12,0.815,28-3,25-9,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,1610612749,0,22022,2022-12-18,East,Milwaukee,29,21,8,0.724,14-3,7-5,NaN
4320,1610612749,0,22022,2022-12-19,East,Milwaukee,30,22,8,0.733,14-3,8-5,NaN
4305,1610612749,0,22022,2022-12-20,East,Milwaukee,30,22,8,0.733,14-3,8-5,NaN
4290,1610612749,0,22022,2022-12-21,East,Milwaukee,31,22,9,0.710,14-3,8-6,NaN
